![image](https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/notebooks/headers/watsonx-Prompt_Lab-Notebook.png)
# Use watsonx, and `mistralai/mistral-large` to make simple chat conversation and tool calls

#### Disclaimers

- Use only Projects and Spaces that are available in watsonx context.


## Notebook content

This notebook provides a detailed demonstration of the steps and code required to showcase support for Chat models, including the integration of tools and watsonx.ai models.

Some familiarity with Python is helpful. This notebook uses Python 3.11.


## Learning goal

The purpose of this notebook is to demonstrate how to use Chat models, e.g. `mistralai/mistral-large` by using tools.


## Table of Contents

This notebook contains the following parts:

- [Setup](#setup)
- [Foundation Models on watsonx](#models)
- [Work with chat messages](#chat)
- [Summary](#summary)

<a id="setup"></a>
## Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Create a <a href="https://cloud.ibm.com/catalog/services/watsonxai-runtime" target="_blank" rel="noopener no referrer">watsonx.ai Runtime Service</a> instance (a free plan is offered and information about how to create the instance can be found <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/getting-started/wml-plans.html?context=wx&audience=wdp" target="_blank" rel="noopener no referrer">here</a>).

### Install and import the `datasets` and dependencies

In [2]:
!uv pip install -U "langchain_ibm>=0.3,<0.4"

Using Python 3.12.12 environment at: C:\Users\koste\Desktop\UW\Arbitraz\.venv
Resolved 40 packages in 875ms
Prepared 14 packages in 2.54s
Uninstalled 6 packages in 1.00s
Installed 14 packages in 1.55s
 - cachetools==6.2.1
 + cachetools==6.2.2
 - certifi==2025.10.5
 + certifi==2025.11.12
 - ibm-watsonx-ai==1.4.2
 + ibm-watsonx-ai==1.4.7
 + jsonpatch==1.33
 + langchain-core==0.3.80
 + langchain-ibm==0.3.20
 + langsmith==0.4.46
 - numpy==1.26.4
 + numpy==2.3.5
 + orjson==3.11.4
 - pydantic==2.12.3
 + pydantic==2.12.4
 - pydantic-core==2.41.4
 + pydantic-core==2.41.5
 + requests-toolbelt==1.0.0
 + tenacity==9.1.2
 + zstandard==0.25.0


### Define the watsonx.ai credentials
Use the code cell below to define the watsonx.ai credentials that are required to work with watsonx Foundation Model inferencing.

**Action:** Provide the IBM Cloud user API key. For details, see <a href="https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui" target="_blank" rel="noopener no referrer">Managing user API keys</a>.

In [3]:
import getpass
from ibm_watsonx_ai import Credentials

credentials = Credentials(
    url="https://eu-gb.ml.cloud.ibm.com",
    api_key=getpass.getpass("Enter your watsonx.ai api key and hit enter: "),
)

### Define the project ID
You need to provide the project ID to give the Foundation Model the context for the call. If you have a default project ID set in Watson Studio, the notebook obtains that project ID. Otherwise, you need to provide the project ID in the code cell below.

In [4]:
import os

try:
    project_id = os.environ["PROJECT_ID"]
except KeyError:
    project_id = input("Enter your project_id and hit enter: ")

<a id="models"></a>
## Set up the Foundation Model on `watsonx.ai`


Specify the `model_id` of the model you will use for the chat with tools.

In [7]:
from ibm_watsonx_ai import Credentials, APIClient
client = APIClient(credentials, project_id)

for model in client.foundation_models.get_model_specs()['resources']:
    print(model["model_id"])

model_id = "ibm/granite-3-8b-instruct"


cross-encoder/ms-marco-minilm-l-12-v2
ibm/granite-3-8b-instruct
ibm/granite-embedding-278m-multilingual
ibm/granite-ttm-1024-96-r2
ibm/granite-ttm-1536-96-r2
ibm/granite-ttm-512-96-r2
ibm/slate-125m-english-rtrvr-v2
ibm/slate-30m-english-rtrvr-v2
intfloat/multilingual-e5-large
meta-llama/llama-3-2-11b-vision-instruct
meta-llama/llama-3-3-70b-instruct
meta-llama/llama-4-maverick-17b-128e-instruct-fp8
mistralai/mistral-small-3-1-24b-instruct-2503
sentence-transformers/all-minilm-l6-v2


### Define the model parameters

You might need to adjust model `parameters` depending on the model you use.

In [8]:
from ibm_watsonx_ai.foundation_models.schema import TextChatParameters

TextChatParameters.show()

+-----------------------+----------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| PARAMETER             | TYPE                                   | EXAMPLE VALUE                                                                                                                                                                                                                                                                   |
+=======================+========================================+============================================================================================================================================================================================================================================================

In [9]:
params = TextChatParameters(
    temperature=1
)

### Initialize the model

Initialize the `ModelInference` class with the previously set parameters.

In [10]:
from ibm_watsonx_ai.foundation_models import ModelInference

model = ModelInference(
    model_id=model_id,
    credentials=credentials,
    project_id=project_id,
    params=params
)

<a id="chat"></a>
## Work with chat messages 

#### Work with a simple chat message

In [11]:
messages = [
    {
        "role": "user", 
        "content": "What is 1 + 1"
    }
]

simple_chat_response = model.chat(messages=messages)

In [12]:
print(simple_chat_response["choices"][0]["message"]["content"])

<|python_start|>def add(a, b):
    return a + b

result = add(1, 1)
print(result)<|python_end|>


#### Work with a simple chat message using chat_stream

In [13]:
messages = [
    {
        "role": "user", 
        "content": "What IBM mainly does?"
    }
]

simple_chat_stream_response = model.chat_stream(messages=messages)

In [14]:
for chunk in simple_chat_stream_response:
    print(chunk["choices"][0]["delta"].get("content", ""), end="", flush=True)

<|python_start|>tool_call(function_name="get_company_info", args={"company_name": "IBM"})<|python_end|>

IndexError: list index out of range

#### Work with an advanced chat message

In [15]:
messages = [
    {
        "role": "system",
        "content": "You are a helpful assistant."
    },
    {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": "Who won the world series in 2020?"
            }
        ],
    },
    {
        "role": "assistant",
        "content": "The Los Angeles Dodgers won the World Series in 2020."
    },
    {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": "Where was it played?"
            }
        ],
    }
]

advanced_chat_response = model.chat(messages=messages)

In [16]:
print(advanced_chat_response["choices"][0]["message"]["content"])

The 2020 World Series was played at Globe Life Field in Arlington, Texas. The series was between the Los Angeles Dodgers and the Tampa Bay Rays. The Dodgers won the series 4 games to 2. Globe Life Field was the home stadium of the Texas Rangers, but it was used as a neutral site due to COVID-19 pandemic restrictions.


#### Work with chat messages using `tools` and `tool_choice`

In [18]:
def get_current_weather(location: str, unit: str = "celsius") -> dict:
    """
    Get the current weather in a given location.

    Parameters:
    - location (str): The city and state, e.g., "San Francisco, CA".
    - unit (str): The unit for temperature, either "celsius" or "fahrenheit". Defaults to "celsius".

    Returns:
    - dict: A dictionary containing the current weather details.
    """
    weather_data = {
        "location": location,
        "temperature": 20,
        "unit": unit,
        "description": "Partly cloudy"
    }
    return weather_data

Converting raw python function to correct tool schema

In [19]:
import json
from langchain_ibm.chat_models import convert_to_openai_tool

formatted_tools = [convert_to_openai_tool(get_current_weather)]
print(json.dumps(formatted_tools, indent=4))

[
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location. Parameters:\n- location (str): The city and state, e.g., \"San Francisco, CA\".\n- unit (str): The unit for temperature, either \"celsius\" or \"fahrenheit\". Defaults to \"celsius\".",
            "parameters": {
                "properties": {
                    "location": {
                        "type": "string"
                    },
                    "unit": {
                        "default": "celsius",
                        "type": "string"
                    }
                },
                "required": [
                    "location"
                ],
                "type": "object"
            }
        }
    }
]


In [20]:
messages = [
    {"role": "user", "content": "What's the weather like in Boston today?"},
]

tool_choice = {"type": "function", "function": {"name": "get_current_weather"}}

tool_response = model.chat(messages=messages, tools=formatted_tools, tool_choice=tool_choice)

In [21]:
print(json.dumps(tool_response["choices"][0]["message"], indent=4))

{
    "role": "assistant",
    "content": "",
    "tool_calls": [
        {
            "id": "chatcmpl-tool-15cd8a04f9744d558d7512de693579b4",
            "type": "function",
            "function": {
                "name": "get_current_weather",
                "arguments": "{\"location\": \"Boston, MA\", \"unit\": \"fahrenheit\"}"
            }
        }
    ]
}


#### Work with chat messages using `tools` and `tool_choice_option`

In [22]:
def add_numbers(a: float, b: float) -> float:
    """
    Adds two numbers.

    Parameters:
    - a (float): The first number.
    - b (float): The second number.

    Returns:
    - float: The result of a + b.
    """
    return a + b


def multiply_numbers(a: float, b: float) -> float:
    """
    Multiplies two numbers.

    Parameters:
    - a (float): The first number.
    - b (float): The second number.

    Returns:
    - float: The result of a * b.
    """
    return a * b

Converting raw python function to correct tool schema

In [23]:
tool_choice_option = "auto"

formatted_tools = [convert_to_openai_tool(tool) for tool in [add_numbers, multiply_numbers]]

print(json.dumps(formatted_tools, indent=4))

[
    {
        "type": "function",
        "function": {
            "name": "add_numbers",
            "description": "Adds two numbers. Parameters:\n- a (float): The first number.\n- b (float): The second number.",
            "parameters": {
                "properties": {
                    "a": {
                        "type": "number"
                    },
                    "b": {
                        "type": "number"
                    }
                },
                "required": [
                    "a",
                    "b"
                ],
                "type": "object"
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "multiply_numbers",
            "description": "Multiplies two numbers. Parameters:\n- a (float): The first number.\n- b (float): The second number.",
            "parameters": {
                "properties": {
                    "a": {
                        "type": "number"
     

In [24]:
messages = [
    {"role": "user", "content": "What is 5 + 6?"},
]

tools_response = model.chat(messages=messages, tools=formatted_tools, tool_choice_option=tool_choice_option)

print(json.dumps(tools_response["choices"][0]["message"], indent=4))

{
    "role": "assistant",
    "tool_calls": [
        {
            "id": "chatcmpl-tool-5da24e84fd39479a892e67c01271031a",
            "type": "function",
            "function": {
                "name": "add_numbers",
                "arguments": "{\"a\": 5, \"b\": 6}"
            }
        }
    ]
}


In [25]:
messages = [
    {"role": "user", "content": "What is 5 * 6?"},
]

tools_response_2 = model.chat(messages=messages, tools=formatted_tools, tool_choice_option=tool_choice_option)

print(json.dumps(tools_response_2["choices"][0]["message"], indent=4))

{
    "role": "assistant",
    "tool_calls": [
        {
            "id": "chatcmpl-tool-b60215a7fda94f9ba14be436dce804a9",
            "type": "function",
            "function": {
                "name": "multiply_numbers",
                "arguments": "{\"a\": 5, \"b\": 6}"
            }
        }
    ]
}


#### Execute tool calls

We organize the two functions into a dictionary where keys represent the function name, and values are the function.

In [26]:
names_to_functions = {
    "add_numbers": add_numbers,
    "multiply_numbers": multiply_numbers,
}

In [27]:
tool_call = tools_response["choices"][0]["message"]["tool_calls"]
function_name = tool_call[0]["function"]["name"]
function_params = json.loads(tool_call[0]["function"]["arguments"])
print(f"Executing function: `{function_name}`, with parameters: {function_params}")

function_result = names_to_functions[function_name](**function_params)
print(f"Function result: {function_result}")

Executing function: `add_numbers`, with parameters: {'a': 5, 'b': 6}
Function result: 11


In [28]:
tool_call = tools_response_2["choices"][0]["message"]["tool_calls"]
function_name = tool_call[0]["function"]["name"]
function_params = json.loads(tool_call[0]["function"]["arguments"])
print(f"Executing function: `{function_name}`, with parameters: {function_params}")


function_result = names_to_functions[function_name](**function_params)
print(f"Function result: {function_result}")

Executing function: `multiply_numbers`, with parameters: {'a': 5, 'b': 6}
Function result: 30


<a id="summary"></a>
## Summary and next steps

You successfully completed this notebook!

You learned how to work with chat models using tools and watsonx.ai.

Check out our _<a href="https://ibm.github.io/watsonx-ai-python-sdk/samples.html" target="_blank" rel="noopener no referrer">Online Documentation</a>_ for more samples, tutorials, documentation, how-tos, and blog posts. 

### Author

**Mateusz Szewczyk**, Software Engineer at watsonx.ai.

Copyright © 2024-2025 IBM. This notebook and its source code are released under the terms of the MIT License.